In [1]:
import pandas as pd 
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('watchGo').getOrCreate()
sqlContext = SQLContext(spark)

import warnings
warnings.filterwarnings("ignore")

In [2]:
def df_hdfs_spark(str_, csv, transform = False):
    '''
    Lee unos datos, los guarda en ficheros de HDFS y devuelve un
    dataframe de spark
    Parámtros: un nombre para el .parquet y un csv. 
    El parámetro transform es especial para el fichero de filmaffinity
    '''
    if transform:
        data = pd.read_csv('data/' + csv)
        for i in range(len(data)):
            data.iloc[i,1] = data.iloc[i,1].replace('[', '').replace(']', '').replace("\'", '').split(',')
        df_spark = sqlContext.createDataFrame(data=data) #pd.dataframe --> spark
    else:
        df_spark = spark.read.format('csv').options(header = 'true').load('data/' + csv)
    
    path = 'hdfs://namenode:8020/datalake' + str_ + '.parquet'
    df_spark.write.parquet(path, mode = 'overwrite') #spark --> hdfs  ojo con el parametro mode!!
    return spark.read.parquet(path) #hdfs --> spark

In [10]:
# Filmaffinity
fm = df_hdfs_spark('fm', 'filmaffinity.csv', transform=True)
fm.where("Pelicula like '%4 días%' ").collect()

[Row(Pelicula='4 días ', Cines=['Cines La Vaguada', ' Cinesa Las Rozas - Heron City', ' Cinesa Manoteras', ' Cinesa Méndez Álvaro', ' Conde Duque Auditorio Morasol', ' Kinépolis Madrid Ciudad de la Imagen', ' mk2 Palacio de Hielo (antiguos Cines Dreams)', ' Renoir Cines Princesa', ' Yelmo Cines Rivas H2O'])]

In [9]:
# Just Watch
jw = df_hdfs_spark('jw', csv = 'just-watch.csv')
jw.where('Titulo like "%stranger%"').show() 

+----------+--------+--------------------+
|Plataforma|    Tipo|              Titulo|
+----------+--------+--------------------+
|   netflix|   serie|     stranger things|
|   netflix|pelicula|two distant stran...|
|   netflix|   serie|        the stranger|
+----------+--------+--------------------+



In [12]:
# IMDb
im = df_hdfs_spark('im', csv = 'IMDb movies.csv')
im.select(['title', 'genre']).where('title like "%Inception%"').show()

+---------+--------------------+
|    title|               genre|
+---------+--------------------+
|Inception|Action, Adventure...|
+---------+--------------------+



In [41]:
def search_movie(movie):
    print(fm.where('Pelicula like "%' + movie + '%"').collect())
    jw.where('Titulo like "%' + movie + '%"').show()
    im.select(['title', 'genre']).where('title like "%' + movie + '%"').show()

In [43]:
search_movie('Interstellar')

[]
+----------+----+------+
|Plataforma|Tipo|Titulo|
+----------+----+------+
+----------+----+------+

+-----------------+--------------------+
|            title|               genre|
+-----------------+--------------------+
|     Interstellar|Adventure, Drama,...|
|Interstellar Wars|              Sci-Fi|
+-----------------+--------------------+



In [44]:
search_movie('interstellar')

[]
+------------------+--------+------------+
|        Plataforma|    Tipo|      Titulo|
+------------------+--------+------------+
|amazon-prime-video|pelicula|interstellar|
|           netflix|pelicula|interstellar|
|               hbo|pelicula|interstellar|
+------------------+--------+------------+

+-----+-----+
|title|genre|
+-----+-----+
+-----+-----+



In [9]:
spark.stop()